In [29]:
import numpy as np 
import pandas as pd 

#reading data
import pickle

#Data visutalisation 
import matplotlib.pyplot as plt 
import seaborn as sns 

#Tensorflow, teras 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Concatenate , Input
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPool2D,AveragePooling2D,MaxPooling2D,BatchNormalization
from sklearn.metrics import  accuracy_score as score
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical

# for Texts 
import sklearn.feature_extraction.text as text
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer 
import re

#dim reduction
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm as tqdm

GPU=False

def cosine_sim(X2,batch_size,N_neighbor):
    #X2=cupy.asarray(X)
    df_cos_dist=pd.DataFrame(columns=np.arange(N_neighbor))
    df_id=pd.DataFrame(columns=np.arange(N_neighbor))
    X2_T=X2.T
    a=0
    b=batch_size
    n_it=int(len(df)/batch_size)

    for i in tqdm(range(n_it)):
        df_cos_dist_batch=(1-X2[a:b]@X2_T)

        args=np.argsort(df_cos_dist_batch)[:,0:N_neighbor]
        df_cos_dist_batch=np.sort(df_cos_dist_batch)[:,0:N_neighbor]
        

        df_dist=pd.DataFrame(df_cos_dist_batch)
        arg_df=pd.DataFrame(args)

        df_cos_dist = pd.concat([df_cos_dist,df_dist],ignore_index=True)
        df_id = pd.concat([df_id,arg_df],ignore_index=True)

        a+=batch_size
        b+=batch_size

    df_cos_dist_batch=(1-X2[a:]@X2_T)
    args=np.argsort(df_cos_dist_batch)[:,0:N_neighbor]
    df_cos_dist_batch=np.sort(df_cos_dist_batch)[:,0:N_neighbor]


    df_dist=pd.DataFrame(df_cos_dist_batch)
    arg_df=pd.DataFrame(args)

    df_cos_dist = pd.concat([df_cos_dist,df_dist],ignore_index=True)
    df_id = pd.concat([df_id,arg_df],ignore_index=True)
    return np.array(df_cos_dist),np.array(df_id)


if GPU:
    import cupy 
    def cosine_sim(X,batch_size,N_neighbor):
        X2=cupy.asarray(X)
        df_cos_dist=pd.DataFrame(columns=np.arange(N_neighbor))
        df_id=pd.DataFrame(columns=np.arange(N_neighbor))

        a=0
        b=batch_size
        n_it=int(len(df)/batch_size)

        for i in tqdm(range(n_it)):
            df_cos_dist_batch=(1-cupy.matmul(X2,X2[a:b].T).T)

            args=cupy.argsort(df_cos_dist_batch)[:,0:N_neighbor]
            df_cos_dist_batch=cupy.sort(df_cos_dist_batch)[:,0:N_neighbor]

            df_cos_dist_batch=cupy.asnumpy(df_cos_dist_batch)
            args=cupy.asnumpy(args)

            df_dist=pd.DataFrame(df_cos_dist_batch)
            arg_df=pd.DataFrame(args)

            df_cos_dist = pd.concat([df_cos_dist,df_dist],ignore_index=True)
            df_id = pd.concat([df_id,arg_df],ignore_index=True)

            a+=batch_size
            b+=batch_size

        df_cos_dist_batch=(1-cupy.matmul(X2,X2[a:].T).T)
        args=cupy.argsort(df_cos_dist_batch)[:,0:N_neighbor]
        df_cos_dist_batch=cupy.sort(df_cos_dist_batch)[:,0:N_neighbor]

        df_cos_dist_batch=cupy.asnumpy(df_cos_dist_batch)
        args=cupy.asnumpy(args)

        df_dist=pd.DataFrame(df_cos_dist_batch)
        arg_df=pd.DataFrame(args)

        df_cos_dist = pd.concat([df_cos_dist,df_dist],ignore_index=True)
        df_id = pd.concat([df_id,arg_df],ignore_index=True)
        return np.array(df_cos_dist),np.array(df_id)

df = pd.read_csv('../input/shopee-product-matching/train.csv')








def th_len_df(len_of_df):
    return -1.768e-06*len_of_df+0.6269




train=True
if train==True:
    df = pd.read_csv('../input/shopee-product-matching/train.csv')
if train==False:
    df = pd.read_csv('../input/shopee-product-matching/test.csv')

In [13]:
if train:
    N_g_sample=False 
    if N_g_sample:

        N_g=100
        unique_label_group=df.label_group.unique()
        inds=np.arange(len(unique_label_group))
        np.random.seed(seed=50)
        np.random.shuffle(inds)

        inds=inds[0:N_g]

        unique_label_group=unique_label_group[inds]


        L=[]
        for i in range(N_g):
            L.extend(list(df.loc[df.label_group==unique_label_group[i]].index.values))

        np.random.shuffle(L)


        df=df.iloc[L]
        df=df.reset_index(drop=True)



In [14]:
if train:
    a=df.groupby('label_group').count()['image'].mean()
    print(a)
#avg=3.109678590884329
N_groups=4096

3.109678590884329


In [15]:
# Cleaning the data  for the titles 

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# to remove the stop words
stop_words = stopwords.words("english")


stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer() 

#preprocess fontion 
def preprocess(text, stem=False,lem=True):
    # Remove link and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            if lem:
                tokens.append(lemmatizer.lemmatize(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

# cleaning the data for every rows
df.title= df.title.apply(lambda x: preprocess(x))

# TF-Idf
#---------------------------------------------------
#tfidf = text.TfidfVectorizer(max_features=5000) #ok 
#tfidf = text.TfidfVectorizer(max_features=10000) #ok 
tfidf = text.TfidfVectorizer(min_df=2) #ok 


tfidf.fit(df.title)
t = tfidf.transform(df.title)
#just making a matrix 
matrice=pd.DataFrame(t.todense())
matrice.columns=tfidf.get_feature_names()

t=(t.todense())
#---------------------------------------------------

In [16]:
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from keras.layers import Conv2D, MaxPooling2D, InputLayer, Flatten,    Dense, BatchNormalization, AveragePooling2D
from keras.models import Sequential




WGT='../input/effnet/efficientnetb0.h5'

pretrained_model = EfficientNetB0(weights=WGT, include_top=True)
feat_extractor = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer("top_dropout").output)

#shape of the loaded image for the pretrained model 
im_input_shape=feat_extractor.output.shape[1:]

#the shape is the number of columns of he TF-IDF matrix
tfidf_input_shape=t.shape[1]

In [17]:
Im_input_shape=list(pretrained_model.input.shape)[1:]
print(Im_input_shape)

[224, 224, 3]


In [18]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Concatenate


x_im_feat = Input(shape=im_input_shape, name="Images-data")
tfifd_input = Input(shape=tfidf_input_shape, name="tf-idf-data")


x = Dense(N_groups , activation='relu')(x_im_feat)

x_Tf_idf = Dense(N_groups , activation='relu')(tfifd_input)

x=x*x_Tf_idf

model = Model(inputs=[x_im_feat,tfifd_input], outputs=x, name='test')

model.summary()

Model: "test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Images-data (InputLayer)        [(None, 1280)]       0                                            
__________________________________________________________________________________________________
tf-idf-data (InputLayer)        [(None, 14922)]      0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4096)         5246976     Images-data[0][0]                
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 4096)         61124608    tf-idf-data[0][0]                
_______________________________________________________________________________________________

In [19]:
DATADIR="../input/shopee-product-matching/"

#new columns that has the dir + the name of the image
if train:
    df['Im_dir'] = DATADIR + 'train_images/' + df['image']
if train==False:
    df['Im_dir'] = DATADIR + 'test_images/' + df['image']
    
batch_size=500
n_it=int(len(df)/batch_size)
a=0
b=batch_size
batch_list=[]
df2=pd.DataFrame(columns=np.arange(model.output.shape[1]))

for i in range(n_it):
    batch_list.append(df[a:b])
    a+=batch_size
    b+=batch_size
batch_list.append(df[a:])

for i in tqdm(range(len(batch_list))):
    df_batch=batch_list[i]
    #the size of the image

    #Empty list 
    data_im=[]

    IM_SIZE=Im_input_shape[0]
    COLOR_MODE='rgb' 
    #the directory for all individual images
    images=df_batch.Im_dir.values

    #loop over batch of the images 
    for im in (images):

        #reshaping the data into a IM_SIZE X IM_SIZE image
        img_array=tf.keras.preprocessing.image.load_img(im,target_size=(IM_SIZE,IM_SIZE),color_mode=COLOR_MODE)
        img_array = tf.keras.preprocessing.image.img_to_array(img_array)
        
        #Normalization
        #normalizing the image gives us a worste result 
        #img_array=img_array/255.0 
        
        data_im.append(img_array)
    data_im_arr=np.array(data_im)

    ###################################################################
    pretrained_arr=feat_extractor.predict(data_im_arr)

    T=t[batch_list[i].index]
    
    pred_n=model.predict([pretrained_arr,T])
    
    df1=pd.DataFrame(pred_n)
    
    df2 = pd.concat([df2,df1],ignore_index=True)


  0%|          | 0/69 [00:00<?, ?it/s]

In [20]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import Normalizer
transformer = Normalizer(norm='l2').fit(df2)  # fit does nothing.

transformer
if len(df)>3:
    N_neighbor=50
    N_bach=500
    
if len(df)<50:
    N_neighbor=1
    N_bach=1
    
X=transformer.transform(df2)
Knn=cosine_sim(X,N_bach,N_neighbor)

distances=Knn[0]
indices=Knn[1]

  0%|          | 0/68 [00:00<?, ?it/s]

In [21]:
if train:
    n=99
    display(Knn[0][n])
    display(Knn[1][n])
    pred=Knn[1][n]


    dn=df.loc[df.label_group==df.iloc[n].label_group]
    arr=dn.index
    print('N=',len(arr))
    for p in pred:
        print(p in arr)



array([5.3644180e-07, 4.0718979e-01, 5.0870097e-01, 5.2349746e-01,
       5.4362530e-01, 5.4950738e-01, 5.5635583e-01, 5.5961215e-01,
       5.6538320e-01, 5.8138454e-01, 5.9051549e-01, 5.9678829e-01,
       6.0401273e-01, 6.0689139e-01, 6.0903347e-01, 6.1490273e-01,
       6.1807609e-01, 6.2834013e-01, 6.2938416e-01, 6.6281438e-01,
       6.6284931e-01, 6.6861010e-01, 6.7649293e-01, 6.7675841e-01,
       6.7942882e-01, 6.8585086e-01, 6.8633306e-01, 6.9005597e-01,
       6.9168711e-01, 6.9557828e-01, 7.0170110e-01, 7.0204604e-01,
       7.0356756e-01, 7.0965964e-01, 7.1188807e-01, 7.1228564e-01,
       7.1514010e-01, 7.1902949e-01, 7.2242498e-01, 7.2373176e-01,
       7.2435617e-01, 7.3952162e-01, 7.4167979e-01, 7.4334234e-01,
       7.4428868e-01, 7.4448955e-01, 7.4508679e-01, 7.4977744e-01,
       7.5000143e-01, 7.5015813e-01], dtype=float32)

array([99, 27075, 33322, 8668, 26483, 6734, 63, 75, 98, 12393, 2608,
       13476, 21204, 14491, 24157, 31721, 17811, 15767, 7526, 11586, 2517,
       32578, 12932, 22170, 33423, 6201, 28282, 2457, 1186, 32423, 12706,
       11757, 28489, 16143, 21094, 27146, 18326, 14544, 2289, 30531,
       12688, 13259, 21873, 13473, 82, 10256, 28909, 3168, 30319, 4300],
      dtype=object)

N= 41
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
True
False
False
True
True
True
True
False
True
False
False
False
False
False
False
False
False
False


In [30]:
if train:
    tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
    df['matches'] = df['label_group'].map(tmp)
    df['matches'] = df['matches'].apply(lambda x: ' '.join(x))


In [23]:
if train:
    # Function to get our f1 score
    def f1_score(y_true, y_pred):
        y_true = y_true.apply(lambda x: set(x.split()))
        y_pred = y_pred.apply(lambda x: set(x.split()))
        intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
        len_y_pred = y_pred.apply(lambda x: len(x)).values
        len_y_true = y_true.apply(lambda x: len(x)).values
        f1 = 2 * intersection / (len_y_pred + len_y_true)
        return f1

In [32]:
if train:
    predictions = []
    threshold=th_len_df(len(df))


    for k in range(df2.shape[0]):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    score = df['f1'].mean()
    print(score)

0.7313481664692026


## mean F1 score of 0.731 with train Data 

In [25]:
if train:
    #optimizing the threshold
    thresholds=np.arange(0.56,0.60,0.001)
    s=[]
    for th in tqdm(thresholds):
        predictions = []

        indices=Knn[1]
        for k in range(df2.shape[0]):
            idx = np.where(distances[k,] < th)[0]
            ids = indices[k,idx]
            posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
            predictions.append(posting_ids)
        df['pred_matches'] = predictions
        df['f1'] = f1_score(df['matches'], df['pred_matches'])
        score = df['f1'].mean()
        s.append(score)
    s=np.array(s)
    print(thresholds[np.argmax(s)])

  0%|          | 0/40 [00:00<?, ?it/s]

0.561


In [26]:
th_len_df(len(df))

0.566346

In [27]:
predictions = []
threshold=th_len_df(len(df))

indices=Knn[1]
distances=Knn[0]
for k in range(df2.shape[0]):
    idx = np.where(distances[k,] < threshold)[0]
    ids = indices[k,idx]
    posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
    predictions.append(posting_ids)
df['matches'] = predictions

In [28]:
df[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()

,posting_id,matches
0,train_129225211,train_129225211 train_2278313361
1,train_3386243561,train_3386243561 train_3423213080 train_380550...
2,train_2288590299,train_2288590299 train_3803689425
3,train_2406599165,train_2406599165 train_3576714541 train_352677...
4,train_3369186413,train_3369186413 train_921438619
